# Схемы разделения секрета

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime, getPrime
from Crypto.Random import random

# Утилитки
from functools import reduce

In [4]:
def prod(lst):
    """
    Произведение всех элементов списка.
    """
    
    return reduce((lambda x, y: x * y), lst)

----

In [5]:
class GenericParticipant(object):
    def __init__(self, x, y, t):
        """
        Инициализация участника разделения секрета.
        Сохраняются х, у и t.
        """
        
        self._x = x
        self._y = y
        self._t = t
        
        trace('[GenericParticipant]', f'{x = }, {y = }')
        self._verify()
        
        
    def __eq__(self, other):
        """
        Сравнение на равенство. Сравниваю по х - достаточно.
        """
        
        return self.x == other.x
        
        
    @property
    def x(self):
        """
        Получение х.
        """
        
        return self._x
    
    
    @property
    def y(self):
        """
        Получение у.
        """
        
        return self._y
        
        
    @property
    def equation(self):
        """
        Получение соответствующего участнику уравнения.
        """
        
        return [1] + [self._x ^ i for i in range(1, self._t)], self._y
    
    
    def _verify(self):
        pass

In [6]:
class GenericDealer(object):
    def __init__(self, t, n, field):
        """
        Инициализация дилера для разделения секрета.
        Генерация простого числа, поля и ключа.
        """
        
        self._field = field
        self._t = t
        self._n = n
        self._key = self._field.random_element()
        
        self._coefs = []
        self._values = []
        
        trace('[GenericDealer]', f't = {self._t}, n = {self._n}')
        trace('[GenericDealer]', self._field)
        trace('[GenericDealer]', f'key = {self._key}')
    
    
    @property
    def field(self):
        """
        Получение поля.
        """
        
        return self._field
    
    
    @property
    def key(self):
        """
        Получение ключа для проверки
        """
        
        return self._key
    
    
    def _prepare_coefficients(self):
        self._coefs  = [self._field.random_element() for _ in range(self._t - 1)]
        
        self._values = [self._key + sum([self._coefs[i] * x ^ (i + 1) for i in range(self._t - 1)]) 
                        for x in range(1, self._n + 1)]

----

In [7]:
def restore_secret_by_equation_system(field, participants: list[GenericParticipant]):
    """
    Восстановление секрета путем решения системы уравнений.
    """
    
    equations = [p.equation for p in participants]
    M = Matrix(field, [eq[0] for eq in equations])
    v = vector(field, (eq[1] for eq in equations))
    
    return M.solve_right(v)[0]  # Нас интересует только первая переменная,
                                # так как именно она является ключом

In [8]:
def restore_secret_by_lagrange_interpolation(participants: list[GenericParticipant]):
    """
    Восстановление секрета путем интерполяции Лагранжа.
    """
    
    return sum([p1.y * prod([p2.x * (p2.x - p1.x).inverse_of_unit() 
                             for p2 in participants if p1 != p2])
                for p1 in participants])

----

## Схема Шамира
![Shamir scheme](./images/Shamir_1_5.png)
![Shamir scheme](./images/Shamir_2_5.png)

In [9]:
class ShamirParticipant(GenericParticipant):
    def __init__(self, x, y, t):
        """
        Тут в целом все повторяет общий случай.
        """
        
        trace('[ShamirParticipant]', f'Initializing superclass...')
        super(ShamirParticipant, self).__init__(x, y, t)

In [10]:
class ShamirDealer(GenericDealer):
    def __init__(self, t, n):
        """
        Инициализация дилера для разделения секрета.
        Генерация простого числа, поля и ключа.
        """
        
        trace('[ShamirDealer]', f'Initializing superclass...')
        super(ShamirDealer, self).__init__(t, n, GF(getStrongPrime(512)))
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._prepare_coefficients()
        return [ShamirParticipant(self._field.gen() + i, self._values[i], self._t) 
                for i in range(self._n)]

----

In [11]:
t, n = 3, 5

In [12]:
d = ShamirDealer(t, n)

[TRACE] [ShamirDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 11551124262234329248847842442660167982987876083418225564905205197164453837366138976213540459405455278909534187848718780627701362424789259800122871653415833
[TRACE] [GenericDealer] key = 1370112471679205040169219573574876467370713049141906927139497665855453769179749434231352960114824000458876033094922210383669679414226574267931823264220684


In [13]:
participants = d.split_key()

[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 6103834010646811856880462285574665777669913254366200495714574548462744300516506796452655073112288705935381220747175867370646810821366812788043561924545683
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 1358225508329527597951109227964981426205582516449338176277083202720923454093520507597993896476242741516164601901670328806921710353374705844022380012574474
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 3, y = 10235535489196010761076845286066159378953473002227771098637434022958898904643068520094450349017596665020294552255843155947897102859828773036114020835138723
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 4, y = 9633515428777602848561985574557863669937832544865048132985216614847762977432872881514943511925439918628702696112256787538170263491150494764072741085406764

In [14]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 1370112471679205040169219573574876467370713049141906927139497665855453769179749434231352960114824000458876033094922210383669679414226574267931823264220684
Restored correctly: True


In [15]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 10849442512964096115809815343184350129134243992283062815152065894204565146939493085307316249748334670354597839592681405934371911289358919732064743836516892
Restored correctly: False


In [16]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 1370112471679205040169219573574876467370713049141906927139497665855453769179749434231352960114824000458876033094922210383669679414226574267931823264220684
Restored correctly: True


In [17]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 10849442512964096115809815343184350129134243992283062815152065894204565146939493085307316249748334670354597839592681405934371911289358919732064743836516892
Restored correctly: False


---
## Схема Фельдмана
![Feldman scheme](./images/Feldman_5.png)

In [18]:
class FeldmanParticipant(GenericParticipant):
    def __init__(self, x, y, t, g, check_values):
        """
        Тут в дополнение производится проверка.
        """
        
        self._g = g
        self._check_values = check_values
        
        trace('[FeldmanParticipant]', f'Initializing superclass...')
        super(FeldmanParticipant, self).__init__(x, y, t)
        
        
    def _verify(self):
        expected_value = self._g ^ self.y
        value_to_check = prod([cv ^ (self.x ^ i) for i, cv in enumerate(self._check_values)])
        
        print(expected_value, value_to_check)
        
        if value_to_check != expected_value:
            raise ValueError(f'Wrong value given to the participant with x = {self.x} and y = {self.y}')
            
        trace('[FeldmanParticipant]', 'Value verified')

In [19]:
class FeldmanDealer(GenericDealer):
    def __init__(self, t, n):
        """
        """
        
        self._p, self._q, self._g = FeldmanDealer._generate_parameters()
        
        trace('[FeldmanDealer]', f'Initializing superclass...')
        super(FeldmanDealer, self).__init__(t, n, GF(self._q))
        
        trace('[FeldmanDealer]', f'p = {self._p}')
        trace('[FeldmanDealer]', f'q = {self._q}')
        trace('[FeldmanDealer]', f'g = {self._g}')
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._prepare_coefficients()
        check_values = [self._g ^ c for c in [self._key] + self._coefs]
        
        return [FeldmanParticipant(self._field.gen() + i, self._values[i], self._t, self._g, check_values) 
                for i in range(self._n)]
    
    
    @staticmethod
    def _generate_parameters():
        """
        Генерация параметров протокола (p, q, g).
        """
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = getPrime(Q_LENGTH)
        p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
        
        while p not in Primes():
            p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
            
        #
        # Сгенерирую число, взаимно простое с p
        #
        
        tmp = random.getrandbits(P_LENGTH)
        while gcd(tmp, p) != 1:
            tmp = random.getrandbits(P_LENGTH)
        
        #
        # По малой теореме Ферма такой g будет иметь порядок q
        #
        
        R = Integers(p)
        g = R(tmp) ^ ((p - 1) // q)
        
        F = GF(p)
        return p, q, F.gen() * g

----

In [20]:
d = FeldmanDealer(t, n)

[TRACE] [FeldmanDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 820782780864926425059895602166047911390729404919
[TRACE] [GenericDealer] key = 545967487187649114657522052271307838137678561847
[TRACE] [FeldmanDealer] p = 93685863382784788950029896779773484376180361628619798944210654492436536354603666880544563988245330669837076679498388000488783071715824223595461037589463089571636255087163184275658274889852601448295731176562087863038360326453015370552739475164174656954422473290966542031758918993385338090557863535562982588993
[TRACE] [FeldmanDealer] q = 820782780864926425059895602166047911390729404919
[TRACE] [FeldmanDealer] g = 757785540519044890081012451578404282684489027050634540670864823621165226687470986835462877052747570616755372212788386847239685007326097260869845941273605888711556779916063224655449308653583216088066436338283429958335270426406424171738436152193693550623018619910467468927015107880112263473600899057

In [21]:
participants = d.split_key()

[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 275986452892102295136854426234231765137397963458
46528165770406409970423970204582716961786708761593960725642266943258703437708945867574311239221063812618590737510803470062839044679597118255383551524113534957695124139708550537826298497771296130176354481964710242316161983179058335932254562549814612087635397976391858920475330108529723307495986771163263982657 46528165770406409970423970204582716961786708761593960725642266943258703437708945867574311239221063812618590737510803470062839044679597118255383551524113534957695124139708550537826298497771296130176354481964710242316161983179058335932254562549814612087635397976391858920475330108529723307495986771163263982657
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 43375783875618723512234697061116310897932724822
5965858998986181473701739382890399048204338067389

In [22]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 545967487187649114657522052271307838137678561847
Restored correctly: True


In [23]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 172263834397016635697043083631701650529524964317
Restored correctly: False


In [24]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 545967487187649114657522052271307838137678561847
Restored correctly: True


In [25]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 172263834397016635697043083631701650529524964317
Restored correctly: False
